# **PARALLELISATION 1**

In [16]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
import re, json
from copy import copy, deepcopy
from datetime import datetime
from collections import Counter
from Outils import creer_graph, plus_proche_voisin,nb_noeud_unique_troncon_continu,verif_index, gp_changer_nom_geom
from shapely.wkt import dumps, loads

from Simplifier_Rdpt import creer_dico_noeud_rdpt,simplifier_noeud_rdpt,maj_graph_rdpt, donnees_tot_rd_pt
import Estim_trafic as et
import Comptage as Cpt
import Affectation_pt_comptage as ap
import Voies_Cat4 as v4
from MMM import import_fichiers_mmm

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
#rappel des donnees necessaires
bati_ligne_proche=pd.read_json(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\paralelisation2.json')
bati_ligne_proche['point_proche']=bati_ligne_proche.point_proche.apply(lambda x : loads(x))
resultat=pd.read_json(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\trajet_bati_cat123.json')
resultat.trajet_court=resultat.trajet_court.apply(lambda x : tuple(x))
f=open(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\df_tt_trajet_save.json','r')
retour_dict=json.loads(f.read())
f.close()
dico_df_tt_trajet={k:pd.DataFrame.from_dict(v) for k, v in retour_dict.items()}

In [13]:
with ct.ConnexionBdd('local_otv') as c :
    graph_filaire = gp.read_postgis('select * from linearisation_bm.graph_rhv_complet', c.connexionPsy)
    graph_filaire_vertex=gp.read_postgis('select id,cnt,chk,ein,eout,the_geom as geom from linearisation_bm.graph_rhv_complet_vertices_pgr', c.connexionPsy)

In [14]:
#import du fichier de trafic
gdf_traf_1234=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp')
#importdes voies non connues
voies_nc=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\voie_non_affectees.shp')
voies_nc['longueur']=voies_nc.geometry.apply(lambda x : round(x.length,1))
ensemble_lignes=voies_nc.copy()
#j'ai besoin de cette ligne car les source_target de gdf_traf_1234 sont différents de ceux de ensemble_lignes
gdf_tot_eq_1234=graph_filaire.loc[graph_filaire.ident.isin(gdf_traf_1234.ident.tolist())]
vertex_connus=list(set(gdf_tot_eq_1234.source.tolist()+gdf_tot_eq_1234.target.tolist()))
vertex_impasse=[k for k, v in Counter(graph_filaire.source.tolist()+graph_filaire.target.tolist()).items() if v==1]

In [40]:
#i=resultat.index.max()
liste_bati_erreur=[]
for e,b in enumerate(bati_ligne_proche.ID.tolist()[index_arret:]) : 
    if e%100==0:
        print(f'nb bati traite : {e}, {datetime.now()},{b}')
    ident_proche_bati=str(bati_ligne_proche.loc[bati_ligne_proche['ID']==b].ident.to_numpy()[0])#risque si ident pas trouver d'IndexError
    if ensemble_lignes.loc[ensemble_lignes['ident']==ident_proche_bati].empty :
        resultat.loc[i]=[b,ident_proche_bati,np.NaN,np.NaN,'pas ident proche dans voies nc']
        i+=1
        continue
    point_depart=bati_ligne_proche.loc[bati_ligne_proche['ID']==b].point_proche.values[0]
    trajets=v4.ensemble_trajet(ident_proche_bati,point_depart,ensemble_lignes, vertex_connus, vertex_impasse, graph_filaire_vertex)
    noeud_proche=trajets.noeud_proche
    if ident_proche_bati in resultat.ident.to_numpy() :
        df_find_existant=resultat.loc[(resultat['ident']==ident_proche_bati) & (resultat['noeud_proche']==noeud_proche)]
        if not df_find_existant.empty:
            resultat.loc[i]=[b,ident_proche_bati,noeud_proche,df_find_existant.trajet_court.values[0], 'Deduction_trajet-ident connu_noeud_proche connu',df_find_existant.noeud_cat3.values[0]]
            i+=1
        else : 
            trajets.df_trajet_cat3_grp_OD=dico_df_tt_trajet[ident_proche_bati]
            trajets.maj_longueur_tot(trajets.df_trajet_cat3_grp_OD)
            df_finale=trajets.df_trajet_cat3_grp_OD.loc[trajets.df_trajet_cat3_grp_OD.longueur_tot==trajets.df_trajet_cat3_grp_OD.longueur_tot.min()]
            resultat.loc[i]=[b,ident_proche_bati,noeud_proche,tuple(df_finale.lignes.to_numpy()[0]), 'Deduction_trajet-ident connu_noeud_proche varie',df_finale.points.to_numpy()[0][-1]]
            i+=1
    elif ident_proche_bati in [b for a in resultat.loc[~resultat['trajet_court'].isna()].trajet_court.to_numpy() for b in a] :
        resultat_equi=resultat.loc[resultat.trajet_court.apply(lambda x : ident_proche_bati in x if isinstance(x,tuple) else False)].iloc[0]
        trajet_equi=resultat_equi.trajet_court
        trajet_court=trajet_equi[trajet_equi.index(ident_proche_bati):] 
        resultat.loc[i]=[b,ident_proche_bati,trajets.src,trajet_court, 'Deduction_trajet-ident sur chemin',resultat_equi.noeud_cat3]
        i+=1
        resultat.loc[i]=[b,ident_proche_bati,trajets.tgt,trajet_court, 'Deduction_trajet-ident sur chemin',resultat_equi.noeud_cat3]
        i+=1
    else : 
        try :
            trajets.calculs_trajets()
        except v4.PasDeCheminCat3Error : 
            liste_bati_erreur.append(b)
            continue
        resultat.loc[i]=[b,ident_proche_bati,noeud_proche,trajets.trajet_court, 'Calcul_trajet',trajets.point_arrive]
        i+=1
        dico_df_tt_trajet[ident_proche_bati]=trajets.df_trajet_cat3_grp_OD

nb bati traite : 0, 2020-04-29 09:41:33.763213,BATIMENT0000000257204753
nb bati traite : 100, 2020-04-29 09:42:10.317169,BATIMENT0000000257835147
nb bati traite : 200, 2020-04-29 09:43:22.315897,BATIMENT0000000251397645
nb bati traite : 300, 2020-04-29 09:43:56.245448,BATIMENT0000000251397256
nb bati traite : 400, 2020-04-29 09:46:09.839041,BATIMENT0000000251404262
nb bati traite : 500, 2020-04-29 09:47:05.497849,BATIMENT0000000251397937
nb bati traite : 600, 2020-04-29 09:48:12.087476,BATIMENT0000000251397295
nb bati traite : 700, 2020-04-29 09:49:55.210950,BATIMENT0000000251396928
nb bati traite : 800, 2020-04-29 09:52:12.879654,BATIMENT0000000251373306
nb bati traite : 900, 2020-04-29 09:52:31.839696,BATIMENT0000000251382956
nb bati traite : 1000, 2020-04-29 09:53:01.881932,BATIMENT0000000251381334
nb bati traite : 1100, 2020-04-29 09:53:20.304753,BATIMENT0000000251381416
nb bati traite : 1200, 2020-04-29 09:54:03.543527,BATIMENT0000000251382143
nb bati traite : 1300, 2020-04-29 09:

In [39]:
%%time
#test sur une ligne
ident_proche_bati=str(bati_ligne_proche.loc[bati_ligne_proche['ID']=='BATIMENT0000000257204753'].ident.to_numpy()[0])#risque si ident pas trouver d'IndexError
point_depart=bati_ligne_proche.loc[bati_ligne_proche['ID']=='BATIMENT0000000257204753'].point_proche.values[0]
trajets=v4.ensemble_trajet(ident_proche_bati,point_depart,ensemble_lignes, vertex_connus, vertex_impasse, graph_filaire_vertex)
trajets.calculs_trajets()   

Wall time: 5.33 s


In [35]:
index_arret=bati_ligne_proche.ID.tolist().index(resultat.tail(1).bati.values[0])+1
index_bati_erreur=bati_ligne_proche.ID.tolist()[index_arret]

In [36]:
index_arret

1307

In [37]:
index_bati_erreur

'BATIMENT0000000257204753'

In [41]:
#SAVE RESULTATS FINAUX
resultat.to_json(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\resultats_2.json')

dico_save2={k:v.to_dict() for k, v in dico_df_tt_trajet.items()}
f=open(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\bati\df_tt_trajet_resultats_2.json','w')
f.write(json.dumps(dico_save2))
f.close()

In [1]:
liste_bati_erreur

NameError: name 'liste_bati_erreur' is not defined